# 📱 카카오톡 + 인스타그램 통합 학습

## 순서
1. 환경 설정
2. 데이터 업로드 (CSV + TXT)
3. 데이터 전처리
4. 모델 학습
5. 다운로드

In [ ]:
# ===== 1. 환경 설정 =====
print('🔧 Installing packages...')
!pip install -q transformers datasets peft accelerate bitsandbytes sentencepiece

In [ ]:
# GPU 확인
import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Not available"}')
print(f'CUDA: {torch.cuda.is_available()}')

In [ ]:
# ===== 2. 데이터 파서 클래스 =====

import re
import pandas as pd
from typing import List, Tuple

class UnifiedChatParser:
    """카카오톡 + 인스타그램 통합 파서"""
    
    def __init__(self, your_name: str = "나"):
        self.your_name = your_name
    
    def parse_kakao_csv(self, csv_path: str) -> List[Tuple[str, str]]:
        """카카오톡 CSV 파싱"""
        print(f"\n📱 Parsing KakaoTalk: {csv_path}")
        
        df = pd.read_csv(csv_path)
        pairs = []
        
        prev_msg = None
        prev_sender = None
        
        for _, row in df.iterrows():
            sender = row['User']
            message = str(row['Message']).strip()
            
            if not message or message == 'nan' or len(message) < 1 or len(message) > 200:
                continue
            
            message = self._clean(message)
            
            if prev_msg and prev_sender != sender:
                pairs.append((prev_msg, message))
            
            prev_msg = message
            prev_sender = sender
        
        print(f"  ✅ {len(pairs)} pairs extracted")
        return pairs
    
    def parse_instagram_txt(self, txt_path: str) -> List[Tuple[str, str]]:
        """인스타그램 TXT 파싱"""
        print(f"\n📷 Parsing Instagram: {txt_path}")
        
        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        pairs = []
        messages = []
        
        # 패턴: [이름] 날짜\n\t메시지
        pattern = r'\[([^\]]+)\]\s+(\d{4}년[^\n]+)\n\s+(.+?)(?=\n\[|\Z)'
        
        for match in re.finditer(pattern, content, re.DOTALL):
            sender = match.group(1).strip()
            message = match.group(3).strip()
            
            # 탭, 줄바꿈 정리
            message = ' '.join(message.split())
            message = self._clean(message)
            
            if len(message) < 1 or len(message) > 200:
                continue
            
            messages.append({'sender': sender, 'text': message})
        
        # 대화 쌍 생성
        for i in range(len(messages) - 1):
            if messages[i]['sender'] != messages[i+1]['sender']:
                pairs.append((messages[i]['text'], messages[i+1]['text']))
        
        print(f"  ✅ {len(pairs)} pairs extracted")
        return pairs
    
    def _clean(self, text: str) -> str:
        """텍스트 정리"""
        # URL 제거
        text = re.sub(r'http[s]?://\S+', '', text)
        # 이메일 제거
        text = re.sub(r'\S+@\S+', '', text)
        # 이름 제거 (개인정보)
        text = re.sub(r'삼괴[가-힣]+', '', text)
        # 공백 정리
        text = ' '.join(text.split())
        return text.strip()
    
    def merge_and_deduplicate(self, all_pairs: List[List[Tuple[str, str]]]) -> List[Tuple[str, str]]:
        """병합 및 중복 제거"""
        print("\n🔄 Merging and deduplicating...")
        
        merged = []
        seen = set()
        
        for pairs in all_pairs:
            for inp, out in pairs:
                key = (inp.lower(), out.lower())
                if key not in seen:
                    seen.add(key)
                    merged.append((inp, out))
        
        print(f"  ✅ Total unique pairs: {len(merged)}")
        return merged

print('✅ Parser class loaded')

In [ ]:
# ===== 3. 데이터 업로드 =====

from google.colab import files
import os

os.makedirs('data', exist_ok=True)

print('📤 Upload ALL your chat files (CSV for KakaoTalk, TXT for Instagram):')
print('   You can select multiple files at once!')
uploaded = files.upload()

# 파일 분류
kakao_files = [f for f in uploaded.keys() if f.endswith('.csv')]
instagram_files = [f for f in uploaded.keys() if f.endswith('.txt')]

print(f"\n✅ Uploaded:")
print(f"   📱 KakaoTalk: {len(kakao_files)} files")
print(f"   📷 Instagram: {len(instagram_files)} files")

# 파일 이동
for f in uploaded.keys():
    !mv "{f}" data/

In [ ]:
# ===== 4. 데이터 전처리 =====

parser = UnifiedChatParser(your_name="나")  # 본인 이름으로 변경

all_pairs = []

# 카카오톡 파일 파싱
for csv_file in kakao_files:
    try:
        pairs = parser.parse_kakao_csv(f'data/{csv_file}')
        all_pairs.append(pairs)
    except Exception as e:
        print(f"⚠️ Error: {e}")

# 인스타그램 파일 파싱
for txt_file in instagram_files:
    try:
        pairs = parser.parse_instagram_txt(f'data/{txt_file}')
        all_pairs.append(pairs)
    except Exception as e:
        print(f"⚠️ Error: {e}")

# 병합 및 중복 제거
training_pairs = parser.merge_and_deduplicate(all_pairs)

# 샘플 출력
print("\n📝 Sample conversations:")
for i, (inp, out) in enumerate(training_pairs[:10], 1):
    print(f"{i:2}. User: {inp}")
    print(f"    Bot:  {out}\n")

In [ ]:
# ===== 5. 모델 클래스 정의 =====

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset

class LightweightChatbot:
    def __init__(self, model_name="skt/kogpt2-base-v2"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
    
    def train(self, training_pairs, output_dir="./kakao_instagram_model", epochs=5, batch_size=4):
        print("\n" + "="*60)
        print("🚀 Starting Training")
        print("="*60)
        
        # 토크나이저
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        special_tokens = {'additional_special_tokens': ['<|user|>', '<|bot|>', '<|end|>']}
        self.tokenizer.add_special_tokens(special_tokens)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # 모델 (4bit 양자화)
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=quantization_config,
            device_map="auto"
        )
        
        self.model.resize_token_embeddings(len(self.tokenizer))
        self.model = prepare_model_for_kbit_training(self.model)
        
        # LoRA 설정
        peft_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["c_attn", "c_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )
        
        self.model = get_peft_model(self.model, peft_config)
        self.model.print_trainable_parameters()
        
        # 데이터셋 준비
        training_texts = [f"<|user|>{inp}<|bot|>{out}<|end|>" for inp, out in training_pairs]
        dataset = Dataset.from_dict({'text': training_texts})
        
        def tokenize(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                max_length=256,
                padding='max_length'
            )
        
        tokenized = dataset.map(tokenize, batched=True, remove_columns=['text'])
        
        # 학습 설정
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            gradient_accumulation_steps=4,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=50,
            save_steps=500,
            save_total_limit=2,
            warmup_steps=100,
            gradient_checkpointing=True,
            optim="adamw_torch",
            report_to="none"
        )
        
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized,
            data_collator=DataCollatorForLanguageModeling(self.tokenizer, mlm=False)
        )
        
        # 학습 시작
        print(f"\n📊 Training on {len(training_pairs)} conversation pairs")
        trainer.train()
        
        # 저장
        print(f"\n💾 Saving to {output_dir}")
        trainer.save_model(output_dir)
        self.tokenizer.save_pretrained(output_dir)
        
        print("\n✅ Training Complete!")
        return output_dir

print('✅ Model class loaded')

In [ ]:
# ===== 6. 학습 실행 =====

chatbot = LightweightChatbot()

output_dir = chatbot.train(
    training_pairs=training_pairs,
    output_dir="./kakao_instagram_model",
    epochs=5,           # 데이터 많으면 3-5로
    batch_size=4        # Colab GPU에 맞게 조정
)

In [ ]:
# ===== 7. 간단한 테스트 =====

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

print("🧪 Loading model for testing...")

tokenizer = AutoTokenizer.from_pretrained(output_dir)
base_model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2")
base_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(base_model, output_dir)
model.eval()

def test_generate(user_input):
    prompt = f"<|user|>{user_input}<|bot|>"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            temperature=0.7,
            top_p=0.85,
            repetition_penalty=1.8,
            do_sample=True
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if '<|bot|>' in response:
        response = response.split('<|bot|>')[-1].split('<|end|>')[0].strip()
    return response

# 테스트
test_inputs = ["안녕", "뭐해?", "ㅋㅋㅋ", "오늘 뭐 먹을까?", "게임하자"]

print("\n" + "="*60)
print("📝 Test Results")
print("="*60)

for inp in test_inputs:
    response = test_generate(inp)
    print(f"User: {inp}")
    print(f"Bot:  {response}")
    print("-" * 60)

In [ ]:
# ===== 8. 모델 다운로드 =====

print("📦 Zipping model...")
!zip -r kakao_instagram_model.zip {output_dir}

print("\n⬇️ Downloading...")
files.download('kakao_instagram_model.zip')

print("\n✅ Complete! Extract and use with your local app.py")